In [46]:
import pandas as pd
import numpy as np
import talib
import random
import datetime
import matplotlib.pyplot as plt
import statsmodels.api as sm
import xgboost as xgb
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
def formatBacktestdata(_ccy:str):
    # Formatting
    _tarCol = ['Open time', 'Open', 'High', 'Low', 'Volume', 'Close']
    _Dir = "C:\\Users\\LCYY\\Desktop\\CrytoTrade\\Data\\Binance\\"
    _filename = '{}USDT.csv'.format(_ccy)
    _histData = pd.read_csv(_Dir + _filename, dtype={'Open time':str})[_tarCol].dropna()
    _histData['datetime'] = pd.to_datetime(_histData['Open time'], format='%Y-%m-%d %H:%M:%S')
    _histData = _histData.sort_values('datetime').reset_index(drop=True)
    _histData['date'] = _histData.datetime.dt.date
    _histData['time'] = _histData.datetime.dt.time
    _histData['symbol'] = _ccy + '/' + 'USDT'
    _histData = _histData[['datetime', 'date', 'time', 'symbol'] + _tarCol]
    _histData = _histData.drop(columns=['Open time'], axis=1)
    return _histData

In [3]:
def getRandomPeriod(_date:list, _numDays:int):
    _minDate = datetime.date(2020, 12, 31)
    _maxDate = datetime.date(2021, 10, 31)
    _startPeriod = random.choice(_date)
    _endPeriod = _startPeriod + datetime.timedelta(days=_numDays)
    if (_startPeriod >= _minDate) and (_endPeriod <= _maxDate) :
        return _startPeriod, _endPeriod
    else:
        return getRandomPeriod(_date, _numDays)

In [4]:
def calRSI(_close:pd.Series, _window_length:int,_mode:str):
    delta = _close.diff()[1:]
    up, down = delta.clip(lower=0), delta.clip(upper=0).abs()
    if _mode == 'EWMA':
        # Calculate the RSI based on EWMA
        roll_up = up.ewm(span=_window_length).mean()
        roll_down = down.ewm(span=_window_length).mean()
    elif _mode == 'SMA':
        # Calculate the RSI based on SMA
        roll_up = up.rolling(_window_length).mean()
        roll_down = down.rolling(_window_length).mean()
    elif _mode == 'RMA':
        # Calculate the RSI based on RMA/SMMA/MMA
        alpha = 1 / _window_length
        roll_up = up.ewm(alpha=alpha).mean()
        roll_down = down.ewm(alpha=alpha).mean()
    rs = roll_up / roll_down
    rsi = 100.0 - (100.0 / (1.0 + rs))
    return pd.concat([pd.Series([0]), rsi])

In [5]:
def calSTOSC(_sto:pd.Series, _window_length:int):
    # Stochastic
    _SOSC = []
    for i in range(len(_sto)-_window_length):
        _lookbackPeriod = _sto[i:i+_window_length]
        _localmin, _localmax = _lookbackPeriod.min(), _lookbackPeriod.max()
        if _localmax - _localmin == 0:
            _SOSC.append(0)
        else:
            _SOSC.append((_sto[i] - _localmin)/(_localmax - _localmin)*100)
    _SOSC = pd.Series([0]*_window_length + _SOSC)
    return _SOSC

In [6]:
def calFastSlowK(_sto:pd.Series, _high:pd.Series, _low:pd.Series, _window_length:int, _smoother:int):
    # Stochastic Oscillator
    _localHigh = _high.rolling(_window_length).max()
    _localLow = _low.rolling(_window_length).min()
    _FastK = (_sto - _localLow) * 100 / (_localHigh - _localLow)
    _SlowK = _FastK.rolling(_smoother).mean()
    return _FastK, _SlowK

In [7]:
def calEMA(_close:pd.Series, _window_length:int):
    return _close.ewm(span=_window_length).mean()

In [8]:
def calMACD(_close:pd.Series, _windowK:int, _windowD:int, _windowS:int):
    _kLine = _close.ewm(span=_windowK).mean()
    _dLine = _close.ewm(span=_windowD).mean()
    _macd = _kLine-_dLine
    _macd_s = _macd.ewm(span=_windowS).mean()
    _histo = _macd - _macd_s
    return _macd, _macd_s, _histo

In [9]:
def calSuperTrend(_close:pd.Series, _high:pd.Series, _low:pd.Series, _window_length:int, _multiplier:int):
    # Close cross above SuperTrend = Buy
    # ATR
    TRHL = pd.DataFrame(_high - _low)
    TRHC = pd.DataFrame(abs(_high - _close.shift(1)))
    TRLC = pd.DataFrame(abs(_low - _close.shift(1)))
    TR = pd.concat([TRHL, TRHC, TRLC], axis = 1, join = 'inner').max(axis = 1)
    ATR = TR.ewm(_window_length).mean()
    # H/L AVG AND BASIC UPPER & LOWER BAND
    HLA = (_high + _low) / 2
    BUB = (HLA + _multiplier * ATR).dropna()
    BLB = (HLA - _multiplier * ATR).dropna()
    # FINAL BANDS
    final_bands = pd.DataFrame(columns = ['upper', 'lower'])
    final_bands.iloc[:,0] = [x for x in BUB - BUB]
    final_bands.iloc[:,1] = final_bands.iloc[:,0]
    for i in range(len(final_bands)):
        if i == 0:
            final_bands.iloc[i,0] = 0
            final_bands.iloc[i, 1] = 0
        else:
            # UPPER
            if (BUB[i] < final_bands.iloc[i-1,0]) | (_close[i-1] > final_bands.iloc[i-1,0]):
                final_bands.iloc[i,0] = BUB[i]
            else:
                final_bands.iloc[i,0] = final_bands.iloc[i-1,0]
            # LOWER
            if (BLB[i] > final_bands.iloc[i-1,1]) | (_close[i-1] < final_bands.iloc[i-1,1]):
                final_bands.iloc[i,1] = BLB[i]
            else:
                final_bands.iloc[i,1] = final_bands.iloc[i-1,1]
    # SUPERTREND
    supertrend = pd.DataFrame(columns = [f'supertrend_{_window_length}'])
    supertrend.iloc[:,0] = [x for x in final_bands['upper'] - final_bands['upper']]
    for i in range(len(supertrend)):
        if i == 0:
            supertrend.iloc[i, 0] = 0
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 0] and _close[i] < final_bands.iloc[i, 0]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 0]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 0] and _close[i] > final_bands.iloc[i, 0]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 1]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 1] and _close[i] > final_bands.iloc[i, 1]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 1]
        elif supertrend.iloc[i-1, 0] == final_bands.iloc[i-1, 1] and _close[i] < final_bands.iloc[i, 1]:
            supertrend.iloc[i, 0] = final_bands.iloc[i, 0]
    supertrend = supertrend.set_index(BUB.index)
    supertrend = supertrend.dropna()[1:]
    return pd.Series(supertrend.iloc[:, 0])

In [10]:
def Strat_Volume(_N_volume:float, _threshold:float):
    return 1 if _N_volume >= _threshold else 0

def Strat_FastSlowK(_fastK:float, _slowK:float, _thes:int):
    if ((_fastK < _thes) and (_slowK < _thes) and (_fastK > _slowK)):
        return 1
    elif ((_fastK > (100-_thes)) and (_slowK < (100-_thes)) and (_fastK < _slowK)):
        return -1
    else:
        return 0
    
def Strat_KDSwing(_fastK:float, _slowK:float, _thes:int):
    if ((_fastK < _thes) and (_slowK < _thes) and (abs(_fastK/_slowK-1) < 0.02)):
        return 1
    elif ((_fastK > (100-_thes)) and (_slowK < (100-_thes)) and (abs(_fastK/_slowK-1) < 0.02)):
        return -1
    else:
        return 0
    
def Strat_MACD(_macd:float, _macdS:float, _NmacdH:float):
    if (_macd > _macdS) and (_NmacdH > 0.75):
        return 1
    elif _macd < _macdS and (_NmacdH <= 0.25):
        return -1
    else:
        return 0
    
def Strat_BullishOversold(_close:float, _EMA:float, _ST:float, _SRSI:float, _SOS:float):
    return 1 if (((_SRSI+_SOS)/2 < 20) and ((_close > _EMA) or (_close > _ST))) else 0

def Strat_APO(_APO:pd.Series):    
    #Absolute price oscillator 
    #Buy when APO cross from 0 to +ve
    stoch_signal = []
    signal = 0
    for i in range(len(_APO)):
        if _APO[i] > 0:
            if signal != 1:
                signal = 1
                stoch_signal.append(signal)
            else:
                stoch_signal.append(0)
        elif _APO[i] <= 0:
            if signal != 0:
                signal = 0
                stoch_signal.append(0)
            else:
                stoch_signal.append(0)
        else:
            stoch_signal.append(0)
    return stoch_signal

#Aroon
#IF AROON UP LINE => 70 AND AROON DOWN LINE <= 30 --> BUY SIGNAL
#IF AROON UP LINE <= 30 AND AROON DOWN LINE >= 70 --> SELL SIGNAL
def Strat_AROON(_AROONUP:pd.Series, _AROONDown:pd.Series):
    aroon_signal = []
    signal = 0
    for i in range(len(_AROONUP)):
        if _AROONUP[i] >= 70 and _AROONDown[i] <= 30:
            if signal != 1:
                signal = 1
                aroon_signal.append(signal)
            else:
                aroon_signal.append(0)
        elif _AROONUP[i] <= 30 and _AROONDown[i] >= 70:
            if signal != -1:
                signal = -1
                aroon_signal.append(signal)
            else:
                aroon_signal.append(0)
        else:
            aroon_signal.append(0)
    return aroon_signal

#Money flow index
#MFI < 20 then Buy
def Strat_MFI(_MFI):
    MFI_signal = []
    signal = 0
    for i in range(len(_MFI)):
        if _MFI[i] < 20:
            if signal != 1:
                signal = 1
                MFI_signal.append(signal)
            else:
                MFI_signal.append(0)
        elif _MFI[i] >= 0:
            if signal != 0:
                signal = 0
                MFI_signal.append(0)
            else:
                MFI_signal.append(0)
        else:
            MFI_signal.append(0)
    return MFI_signal

In [11]:
def Gen_backtesting_data(_ccy:str):
    # Load data
    _data = formatBacktestdata(_ccy)
    _startDate, _endDate = getRandomPeriod(_date=_data.date.unique(), _numDays=180)
    _train = _data[(_data.date >= _startDate) & (_data.date <= _endDate)].reset_index(drop=True)
    _test = _data[(_data.date >= datetime.date(2021, 10, 31))].reset_index(drop=True)
#     for data in [_train, _test]:
#         # Normalization
# #         data['SOS'] = calSTOSC(_sto=data['Close'], _window_length=14)
# #         data['SFastK'], data['SSlowK'] = calFastSlowK(_sto=data['Close'], _high=data['High'], _low=data['Low'], \
# #                                                       _window_length=14, _smoother=3)
# #         data['RSI'] = calRSI(data['Close'], _window_length=14, _mode='EWMA')
# #         data['SRSI'] = calSTOSC(_sto=data['RSI'], _window_length=14)
# #         data['SVOL'] = calSTOSC(_sto=data['Volume'], _window_length=14)
# #         data['ST'] = calSuperTrend(_close=data['Close'], _high=data['High'], 
# #                                    _low=data['Low'], _window_length=14, _multiplier=3)
# #         data['MACD'], data['MACD_SLine'], data['MACD_Hist'] = calMACD(_close=data['Close'], _windowK=12, \
# #                                                                 _windowD=26, _windowS=9)
# #         data['N_MACD_Hist'] = ((data.MACD_Hist - data.MACD_Hist.min())/(data.MACD_Hist.max() - data.MACD_Hist.min()))
# #         data['APO'] = talib.APO(data.Close, fastperiod = 4, slowperiod = 6, matype = 0)
# #         data['AROONDown'], data['AROONUP'] = talib.AROON(data['High'], data['Low'], timeperiod=14)
# #         data['MFI'] = talib.MFI(data['High'], data['Low'], data['Close'], data['Volume'], timeperiod=14)
    return _train, _test

In [12]:
def Gen_backtesting_signal(_data:pd.DataFrame):
    # Signal
    _signaldf = pd.DataFrame(data=[x for x in _data.date], columns=['date'])
    _signaldf['time'] = _data.time
    # Volume
    for _vlag in [0, 1, 2, 3, 4, 5]:
        _data['MinMaxVolume_LAG_{}'.format(_vlag)] = ((_data.Volume - _data.Volume.min()) / \
                                          (_data.Volume.max() - _data.Volume.min())).shift(_vlag)
        _limit = _data['MinMaxVolume_LAG_{}'.format(_vlag)].mean() + _data['MinMaxVolume_LAG_{}'.format(_vlag)].std()
        _signaldf['Volume_LAG_{}'.format(_vlag)] = _data['MinMaxVolume_LAG_{}'.format(_vlag)].apply(lambda x: 
            Strat_Volume(_N_volume=x, _threshold=_limit))
    # Oversold
    for _threshold in [20, 25, 30, 40]:
        # Stochastic Oscillator
        _signaldf['Oversold_S.OSC_{}'.format(_threshold)] = _data['SOS'].apply(lambda x : -1 if x \
            <= _threshold else 1 if x >= (100-_threshold) else 0)
        # RSI
        _signaldf['Oversold_RSI_{}'.format(_threshold)] = _data['RSI'].apply(lambda x : -1 if x \
            <= _threshold else 1 if x >= (100-_threshold) else 0)
        # Stochastic RSI
        _signaldf['Oversold_S.RSI_{}'.format(_threshold)] = _data['SRSI'].apply(lambda x : -1 if x \
            <= _threshold else 1 if x >= (100-_threshold) else 0)
        # FastK, SlowK
        _signaldf['Oversold_FastSlowK_{}'.format(_threshold)] = _data[['SFastK', 'SSlowK']].apply(lambda x : 
            Strat_FastSlowK(_fastK = x[0], _slowK = x[1], _thes=_threshold), axis=1)
        _signaldf['Oversold_KD_{}'.format(_threshold)] = _data[['SFastK', 'SSlowK']].apply(lambda x : 
            Strat_KDSwing(_fastK = x[0], _slowK = x[1], _thes=_threshold), axis=1)
    # Bullish Trend
    # Super Trend
    _signaldf['SuperTrend'] = _data[['Close', 'ST']].apply(lambda x : -1 if x[0] <= x[1] else 1, axis=1)
    # MACD
    _signaldf['MACD'] = _data[['MACD', 'MACD_SLine', 'N_MACD_Hist']].apply(lambda x :
                        Strat_MACD(_macd = x[0], _macdS = x[1], _NmacdH = x[2]), axis=1)
    # EMA
    for _ema in [3, 7, 14, 28]:
        _data['EMA{}'.format(_ema)] = _data.Close.ewm(span=_ema).mean()
        _signaldf['EMA_{}'.format(_ema)] = _data[['Close', 'EMA{}'.format(_ema)]].apply(lambda x : 
            -1 if x[0] <= x[1] else 1, axis=1)
    # Mean-Revert & Short-term Momentum
    for _lag in [1, 2, 3, 4, 5, 7, 14, 28]:
        _data['LAG_{}'.format(_lag)] = (_data.Close/_data.Close.shift(_lag)-1)
        _signaldf['MeanRevert_LAG_{}'.format(_lag)] = _data['LAG_{}'.format(_lag)].apply(lambda x : 1 if x < 0 else 0)
        _signaldf['CarryMomentum_LAG_{}'.format(_lag)] = _data['LAG_{}'.format(_lag)].apply(lambda x : 1 if x > 0 else 0)
    # Complicated
    _signaldf['BullishOversold'] = _data[['Close', 'EMA{}'.format(14), 'ST', 'SRSI', 'SOS']].apply(lambda \
                              x:Strat_BullishOversold(x[0], x[1], x[2], x[3], x[4]), axis=1)
    _signaldf['APO'] = Strat_APO(_APO=_data['APO'])
    _signaldf['AROON'] = Strat_AROON(_AROONUP=_data['AROONUP'], _AROONDown=_data['AROONDown'])
    # Y
    _signaldf['Actual_Y'] = (_data['Close'].shift(-1)/_data['Close']-1).apply(lambda x : 1 if x > 0 else 0)
    
    return _signaldf[(_signaldf.date > min(_signaldf.date)) & \
            (_signaldf.date < max(_signaldf.date))].reset_index(drop=True)

In [13]:
def Gen_FeaturesSelection(_train:pd.DataFrame):
    # Feature Selection
    X = _train.drop(['date', 'time', 'Actual_Y'], axis=1)
    y = _train['Actual_Y']
    # define the model
    model = LogisticRegression(solver='lbfgs', max_iter=1000)
    # fit the model
    model.fit(X, y)
    # get importance
    importances_SOL = pd.DataFrame(data={'Attribute': X.columns, 'Importance': model.coef_[0]})
    importances_SOL = importances_SOL.sort_values(by='Importance', ascending=False).reset_index(drop=True)
    return (importances_SOL)

In [14]:
def Gen_backtesting_fitmodel(_train:pd.DataFrame, _test:pd.DataFrame, _model:str):
    # Split X, y
    _trainX = _train.drop(['date', 'time', 'Actual_Y'], axis=1)
    _trainy = _train['Actual_Y']
    _testX = _test.drop(['date', 'time', 'Actual_Y'], axis=1)
    _testy = _test['Actual_Y']
    # Fit
    if _model == 'LOGI':
        model = sm.Logit(_trainy, _trainX)
        logi = model.fit(solver='lbfgs', max_iter=1000)
        print('Training Data')
        display(logi.summary())
        display(confusion_matrix(_trainy, (logi.predict(_trainX) >= 0.5).astype(int)))
        print(classification_report(_trainy, (logi.predict(_trainX) >= 0.5).astype(int)))
        print('Testing Data')
        display(confusion_matrix(_testy, (logi.predict(_testX) >= 0.5).astype(int)))
        print(classification_report(_testy, (logi.predict(_testX) >= 0.5).astype(int)))
    elif _model == 'XGBOOST':
        fix_params = {'objective': 'binary:logistic', 'eval_metric':'mlogloss', \
                      'colsample_bytree': 0.35, 'eta': 0.05, 'max_depth': 5, \
                      'n_estimators': 50, 'subsample': 0.45}
        xg_reg = xgb.XGBClassifier(**fix_params)
        xg_reg.fit(_trainX, _trainy)
        print('Training Data')
        display(confusion_matrix(_trainy, xg_reg.predict(_trainX)))
        print(classification_report(_trainy, xg_reg.predict(_trainX)))
        fig, ax = plt.subplots(figsize=(10,10))
        xgb.plot_importance(xg_reg, max_num_features=10, height=0.5, ax=ax,importance_type='weight')
        plt.show()
        print('Testing Data')
        display(confusion_matrix(_testy, xg_reg.predict(_testX)))
        print(classification_report(_testy, xg_reg.predict(_testX)))

In [ ]:
# Feature Selection
_ccyList = ['BTC', 'ETH', 'SOL', 'AVAX', 'ADA']
_fsList = []
for _ccy in _ccyList:
    _Train, _Test = Gen_backtesting_data(_ccy)
    _Signal_Train, _Signal_Test = Gen_backtesting_signal(_Train), Gen_backtesting_signal(_Test)
    _FS = Gen_FeaturesSelection(_Signal_Train)
    _fsList.append(_FS)
concat_imp = pd.concat(_fsList)
_features_avg = concat_imp.groupby(pd.Grouper(key='Attribute')).mean()
_features_avg = _features_avg.sort_values(by='Importance', ascending=False)
_selectedFeatures = _features_avg[:10].index

In [ ]:
# Model Evaluation
_ccyList = ['BTC', 'ETH', 'SOL', 'AVAX', 'ADA']#, 'BNB', 'DOGE', 'DOT', 'XRP']
for _ccy in _ccyList:
    _Train, _Test = Gen_backtesting_data(_ccy)
    _Signal_Train, _Signal_Test = Gen_backtesting_signal(_Train), Gen_backtesting_signal(_Test)
    _Signal_Train_FS, _Signal_Test_FS = _Signal_Train[_selectedFeatures], _Signal_Test[_selectedFeatures]
    Gen_backtesting_fitmodel(_train=_Signal_Train, _test=_Signal_Test, _model='LOGI')

In [ ]:
#_SOLTrain, _SOLTest = Gen_backtesting_data('SOL')

In [ ]:
#_Signal_SOLTrain, _Signal_SOLTest = Gen_backtesting_signal(_SOLTrain), Gen_backtesting_signal(_SOLTest)
#_Signal_SOLTrain_FS, _Signal_SOLTest_FS = _Signal_SOLTrain[_selectedFeatures], _Signal_SOLTest[_selectedFeatures]

In [ ]:
#Gen_backtesting_fitmodel(_train=_Signal_SOLTrain, _test=_Signal_SOLTest, _model='LOGI')

# Tmp Test

In [112]:
def Gen_backtesting_data(_ccy:str):
    # Load data
    _data = formatBacktestdata(_ccy)
    _startDate, _endDate = getRandomPeriod(_date=_data.date.unique(), _numDays=180)
    _train = _data[(_data.date >= _startDate) & (_data.date <= _endDate)].reset_index(drop=True)
    _test = _data[(_data.date >= datetime.date(2021, 10, 31))].reset_index(drop=True)
    for data in [_train, _test]:
        # Normalization
        # Return Lag
        for _lag in [1, 2, 3, 4, 5, 7, 15, 30]:
            data['Return_LAG_{}'.format(_lag)] = (data.Close/data.Close.shift(_lag)-1)
        # Realized Volatility
        for _rv in [15, 30, 60, 180, 360, 720, 1440]:
            ann_factor = 1440 * 365 / _rv
            data['RealizedVol_Window_{}'.format(_rv)] = np.sqrt(np.square(data.Return_LAG_1).rolling(_rv).sum()*ann_factor)
        # ADTV
        for _adtv in [3, 5, 15, 30, 60, 180, 360, 1440]:
            data['ADTV_{}'.format(_adtv)] = data.Volume.rolling(window=_adtv).mean().shift(1)
        # RSI
        for _rsi in [3, 5, 7, 15, 30, 60, 180, 360, 1440]:
            data['RSI_{}'.format(_rsi)] = calRSI(data.Close, _window_length=_rsi, _mode='EWMA')
    return _train, _test

In [113]:
_SOLTrain, _SOLTest = Gen_backtesting_data('SOL')

In [114]:
def Gen_backtesting_signal_cont(_data:pd.DataFrame):
    # Signal
    _signaldf = pd.DataFrame(data=[x for x in _data.date], columns=['date'])
    _signaldf['time'] = _data.time
#     # SOS
#     _signaldf['SOS'] = _data['SOS']
#     # RSI
#     _signaldf['RSI'] = _data['RSI']
#     _signaldf['SRSI'] = _data['SRSI']
#     # SVOL
#     _signaldf['SVOL'] = _data['SVOL']
#     # APO
#     _signaldf['APO'] = _data['APO']
#     # MFI
#     _signaldf['MFI'] = _data['MFI']
#     # KD
#     _signaldf['KD'] = _data['SFastK'] - _data['SSlowK']
#     # ST
#     _signaldf['SuperTrend'] = _data[['Close', 'ST']].apply(lambda x : -1 if x[0] <= x[1] else 1, axis=1)
    # Return Lag
    for _lag in [1, 2, 3, 4, 5, 7, 15, 30]:
        _signaldf['Return_LAG_{}'.format(_lag)] = _data['Return_LAG_{}'.format(_lag)]
    # Realized Volatility
    RV_scaled = pd.DataFrame(preprocessing.StandardScaler().fit_transform(_data[_data.columns[['RealizedVol' \
                      in col for col in _data.columns]]]), columns=_data[_data.columns[['RealizedVol' in \
                          col for col in _data.columns]]].columns)
    # ADTV
    ADTV_scaled = pd.DataFrame(preprocessing.StandardScaler().fit_transform(_data[_data.columns[['ADTV' \
                      in col for col in _data.columns]]]), columns=_data[_data.columns[['ADTV' in \
                          col for col in _data.columns]]].columns)
    # RSI
    RSI_scaled = pd.DataFrame(preprocessing.StandardScaler().fit_transform(_data[_data.columns[['RSI' \
                      in col for col in _data.columns]]]), columns=_data[_data.columns[['RSI' in \
                          col for col in _data.columns]]].columns)
    # concat
    _signaldf = pd.concat([_signaldf, RV_scaled, ADTV_scaled, RSI_scaled], axis=1)
#     # EMA Diff
#     for _ema in [3, 5, 7, 14, 28, 60, 90, 120, 150, 180]:
#         _signaldf['EMA_{}'.format(_ema)] = _data.Close - _data.Close.ewm(span=_ema).mean()
#     # MACD Hist
#     _signaldf['MACD_Hist'] = _data['MACD_Hist']
    # Y
    _signaldf['Actual_Y'] = (_data['Close'].shift(-1)/_data['Close']-1).apply(lambda x : 1 if x > 0 else 0)
    
    return _signaldf[(_signaldf.date > min(_signaldf.date)) & \
            (_signaldf.date < max(_signaldf.date))].reset_index(drop=True)

In [115]:
# data
_Traindf = Gen_backtesting_signal_cont(_data=_SOLTrain)
_Testdf = Gen_backtesting_signal_cont(_data=_SOLTest)

# VIF

In [116]:
def vifcl(df):
    a = 0
    i = 0
    vif = pd.DataFrame()
    vif["VIF Factor"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    vif["features"] = df.columns
    t = vif["VIF Factor"].max()
    for each in vif["VIF Factor"]:
        if each == t and each >= 10:
            a = i
        i += 1
    vax = vif["VIF Factor"].loc[a]
    print(vif.loc[a],"\n>10")
    return a,vax

In [ ]:
# Raw train/test for VIF
_trainX_VIF = _Traindf.drop(['date', 'time', 'Actual_Y'], axis=1)
_trainy_VIF = _Traindf['Actual_Y']
# First Fit
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(_trainX_VIF.values, i) for i in range(_trainX_VIF.shape[1])]
vif["features"] = _trainX_VIF.columns
vif.round(1)
# Drop VIF>10
dropcindex=[]
while True:
    vmax = 0
    dropt = False
    k,s = vifcl(_trainX_VIF)
    print(s)
    if s >= 10:
        dropt = True
    if dropt == True:
        print("dropped",_trainX_VIF.columns[k])
        dropcindex.append(_trainX_VIF.columns[k])
        _trainX_VIF = _trainX_VIF.drop([_trainX_VIF.columns[k]],axis=1)
        vif = vif.drop([k])
        _trainX_VIF = _trainX_VIF.reset_index(drop=True)
        vif = vif.reset_index(drop=True)
    else:
        break
        
print(dropcindex)

VIF Factor    5335.58
features        RSI_7
Name: 25, dtype: object 
>10
5335.575547354514
dropped RSI_7
VIF Factor    650.451
features       RSI_30
Name: 26, dtype: object 
>10
650.4511226463693
dropped RSI_30
VIF Factor    122.018
features        RSI_5
Name: 24, dtype: object 
>10
122.01762206270597
dropped RSI_5


In [111]:
_trainX_VIF

,Return_LAG_1,Return_LAG_2,Return_LAG_3,Return_LAG_4,Return_LAG_5,Return_LAG_7,Return_LAG_15,Return_LAG_30,RealizedVol_Window_15,RealizedVol_Window_30,...,RealizedVol_Window_720,RealizedVol_Window_1440,ADTV_3,ADTV_5,ADTV_15,ADTV_30,ADTV_60,ADTV_180,ADTV_360,ADTV_1440
0,-0.009711,-0.011895,-0.016659,-0.018601,-0.020009,-0.020009,-0.030301,-0.036862,1.672666,2.002264,...,1.866340,1.475089,-0.410318,-0.238837,-0.206507,-0.067447,0.389490,1.976923,1.681582,0.677216
1,-0.008416,-0.018045,-0.020210,-0.024934,-0.026860,-0.033799,-0.030620,-0.046010,1.695224,2.140078,...,1.866955,1.479605,-0.203152,-0.285786,-0.149873,-0.037677,0.380559,1.954160,1.680443,0.675676
2,-0.000148,-0.008562,-0.018190,-0.020355,-0.025078,-0.028399,-0.026759,-0.039149,1.623801,2.035512,...,1.866889,1.479600,-0.065979,-0.234101,-0.174519,-0.048900,0.371176,1.951537,1.681234,0.676159
3,0.005462,0.005314,-0.003147,-0.012827,-0.015004,-0.021689,-0.021795,-0.030048,1.748263,2.063510,...,1.870077,1.480124,0.231162,-0.026820,-0.203334,-0.078628,0.377036,1.951072,1.682738,0.676129
4,0.018206,0.023767,0.023616,0.015002,0.005145,-0.001907,0.003364,-0.012811,2.734915,2.669098,...,1.906135,1.501405,0.268712,0.145323,-0.155992,-0.077049,0.341281,1.951708,1.677431,0.675378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257152,0.000412,0.000412,0.000147,0.000854,0.001945,0.003186,-0.004188,-0.002640,-0.506986,-0.736448,...,-0.700165,-0.681217,-0.613158,-0.632944,-0.532137,-0.604797,-0.609706,-0.045184,-0.438618,-0.703197
257153,0.000412,0.000824,0.000824,0.000559,0.001266,0.002387,-0.002229,-0.002756,-0.539878,-0.737462,...,-0.701768,-0.681265,-0.543383,-0.600382,-0.526831,-0.596617,-0.610479,-0.058423,-0.437831,-0.703142
257154,0.000500,0.000912,0.001324,0.001324,0.001059,0.002858,-0.000118,-0.001848,-0.575835,-0.736707,...,-0.702339,-0.681243,-0.494152,-0.572131,-0.517835,-0.593333,-0.616565,-0.074195,-0.437015,-0.702972
257155,-0.000264,0.000235,0.000647,0.001059,0.001059,0.001501,0.003095,-0.001291,-0.790054,-0.742335,...,-0.702322,-0.681255,-0.470895,-0.579242,-0.521345,-0.594877,-0.626636,-0.085991,-0.437156,-0.703075


# Model

In [ ]:
# LOGI
# Post VIF fit
_trainX = _Traindf.drop(['date', 'time', 'Actual_Y'] + dropcindex, axis=1)
_trainy = _Traindf['Actual_Y']
_testX = _Testdf.drop(['date', 'time', 'Actual_Y'] + dropcindex, axis=1)
_testy = _Testdf['Actual_Y']
# Constant
#_trainX = sm.add_constant(_trainX)
#_testX = sm.add_constant(_testX)
model = sm.Logit(_trainy, _trainX)
logi = model.fit(solver='lbfgs', max_iter=1000, C=1e9)
print('Training Data')
display(logi.summary())
display(confusion_matrix(_trainy, (logi.predict(_trainX) >= 0.5).astype(int)))
print(classification_report(_trainy, (logi.predict(_trainX) >= 0.5).astype(int)))
print('Testing Data')
display(confusion_matrix(_testy, (logi.predict(_testX) >= 0.5).astype(int)))
print(classification_report(_testy, (logi.predict(_testX) >= 0.5).astype(int)))

In [ ]:
# Post VIF fit
_trainX = _Traindf.drop(['date', 'time', 'Actual_Y'] + dropcindex, axis=1)
_trainy = _Traindf['Actual_Y']
_testX = _Testdf.drop(['date', 'time', 'Actual_Y'] + dropcindex, axis=1)
_testy = _Testdf['Actual_Y']
fix_params = {'objective': 'binary:logistic', 'eval_metric':'mlogloss', \
              'colsample_bytree': 0.35, 'eta': 0.05, 'max_depth': 5, \
              'n_estimators': 500, 'subsample': 0.45, 'use_label_encoder':False}
xg_reg = xgb.XGBClassifier(**fix_params)
xg_reg.fit(_trainX, _trainy)
print('Training Data')
display(confusion_matrix(_trainy, xg_reg.predict(_trainX)))
print(classification_report(_trainy, xg_reg.predict(_trainX)))
fig, ax = plt.subplots(figsize=(10,10))
xgb.plot_importance(xg_reg, max_num_features=10, height=0.5, ax=ax,importance_type='weight')
plt.show()
print('Testing Data')
display(confusion_matrix(_testy, xg_reg.predict(_testX)))
print(classification_report(_testy, xg_reg.predict(_testX)))

In [ ]:
# # XGBoost
# XTrain = _Signal_SOLTrain.drop(['date', 'time', 'Actual_Y'], axis=1)
# yTrain = _Signal_SOLTrain['Actual_Y']
# XTest = _Signal_SOLTest.drop(['date', 'time', 'Actual_Y'], axis=1)
# yTest = _Signal_SOLTest['Actual_Y']

# fix_params = {'objective': 'binary:logistic', 'eval_metric':'mlogloss', \
#               'colsample_bytree': 0.35, 'eta': 0.05, 'max_depth': 5, 'n_estimators': 50, 'subsample': 0.45}

# xg_reg = xgb.XGBClassifier(**fix_params)
# xg_reg.fit(XTrain, yTrain)
# print('Training Data')
# display(confusion_matrix(yTrain, xg_reg.predict(XTrain)))
# print(classification_report(yTrain, xg_reg.predict(XTrain)))
# print('Testing Data')
# display(confusion_matrix(yTest, xg_reg.predict(XTest)))
# print(classification_report(yTest, xg_reg.predict(XTest)))

In [ ]:
# from pactools.grid_search import GridSearchCVProgressBar
# from sklearn.metrics import precision_score, make_scorer

# custom_scorer = make_scorer(precision_score, greater_is_better=True,  pos_label=1)

# X = _Signal_SOLTrain.drop(['date', 'time', 'Actual_Y'], axis=1)
# y = _Signal_SOLTrain['Actual_Y']

# cv_params = {'eta':[0.04, 0.05, 0.06], 
#              'max_depth':[4, 5, 6],
#              'subsample': [0.4, 0.45, 0.5], 
#              'colsample_bytree':[0.3, 0.35, 0.4],
#              'n_estimators':[25, 50, 100]
#             }
# fix_params = {'objective': 'binary:logistic','eval_metric':'mlogloss', 'use_label_encoder':False}

# gscv = GridSearchCVProgressBar(xgb.XGBClassifier(**fix_params), param_grid=cv_params, cv=5, scoring=custom_scorer,
#                                return_train_score=False, verbose=2)
# gscv.fit(X, y)
# gscv.best_params_

In [ ]:
print(1)